In [ ]:
try:
    from google.colab import files
    uploaded = files.upload()

    import zipfile
    with zipfile.ZipFile("organized_cat_dataset.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    print("Dataset unzipped successfully!")
except:
    print("Running outside Google Colab or skipping file upload.")

Saving organized_cat_dataset.zip to organized_cat_dataset.zip
Dataset unzipped successfully!


In [ ]:
import os
import random
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn.metrics import classification_report

In [ ]:
# MobileViT model from timm
!pip install -q timm
import timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.9 MB/s eta 0:00:00


In [ ]:
# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tfm = transforms.Compose([
    transforms.Resize((224, 224)), # resizes images to 224x224 pixels
    transforms.RandomHorizontalFlip(), # randomly flips and rotates images (for training data augmentation)
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(), # converts images to pytorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Class names
class_names = ['alarmed', 'angry', 'calm', 'pleased']

In [ ]:
# Dataset paths (organized by folders)
train_img_dir = "/content/train_images"
val_img_dir = "/content/valid_images"
test_img_dir = "/content/test_images"

In [ ]:
# Load datasets using ImageFolder
train_set = datasets.ImageFolder(root=train_img_dir, transform=tfm)
val_set = datasets.ImageFolder(root=val_img_dir, transform=tfm)
test_set = datasets.ImageFolder(root=test_img_dir, transform=tfm)

In [ ]:
# Load datasets
train_loader = DataLoader(datasets.ImageFolder(train_img_dir, transform=tfm), batch_size=16, shuffle=True)
val_loader = DataLoader(datasets.ImageFolder(val_img_dir, transform=tfm), batch_size=16)
test_loader = DataLoader(datasets.ImageFolder(test_img_dir, transform=tfm), batch_size=16)

In [ ]:
# Load pretrained MobileViT model
model = timm.create_model('mobilevit_s', pretrained=True, num_classes=4)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Training
for epoch in range(10):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(imgs), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss={total_loss:.4f}")

In [ ]:
# --- Simplified Validation (Accuracy and Loss) ---
if val_loader and val_set and len(val_set) > 0:
    print("\nCalculating Validation Metrics...")
    model.eval()
    correct = 0
    total = 0
    val_running_loss = 0.0 # Initialize validation loss
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # Calculate validation loss for this batch
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

            # Get predictions for accuracy calculation
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    if total > 0:
        val_accuracy = 100 * correct / total
        val_epoch_loss = val_running_loss / len(val_set) # Calculate average validation loss
        print(f"Validation Loss: {val_epoch_loss:.4f}")
        print(f"Validation Accuracy: {val_accuracy:.2f}%")
    else:
        print("No data in validation loader to calculate metrics.")
else:
    print("Validation data not loaded or is empty. Skipping validation.")

In [ ]:
# Test Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs).argmax(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))